In [8]:
import pandas as pd 
import numpy as np
import sys
!{sys.executable} -m pip uninstall pyarrow --yes
!{sys.executable} -m pip install pyarrow

Found existing installation: pyarrow 6.0.1
Uninstalling pyarrow-6.0.1:
  Successfully uninstalled pyarrow-6.0.1
  Using cached pyarrow-6.0.1-cp38-cp38-win_amd64.whl (15.5 MB)


In [11]:
def convertParquetToCsv(filename: str):
  path = './dataset/{}'.format(filename)
  file = pd.read_parquet(path, engine="pyarrow")
  filename = filename.split('.')
  file.to_csv('./dataset/{}.csv'.format(filename[0]))


def convertCsvToParquet(filepath: str, parquetPath: str):
    file = pd.read_csv(filepath)
    file.to_parquet(parquetPath)

In [12]:
convertParquetToCsv('logins.parquet4')
df = pd.read_csv('./dataset/logins.csv')
print(df.head())